In [12]:
from os import getcwd, listdir, mkdir
from os.path import join, exists
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import threading
import warnings

In [13]:
warnings.filterwarnings("ignore")
matplotlib.use('Agg')

In [14]:
def print_dtypes(location):
    frame = pd.read_csv(location)
    print(frame.dtypes.value_counts())
    print()

In [15]:
data_location = join(getcwd(), "data")
csv_location = join(data_location, "csv")

In [16]:
files = [(folder, file_location) for folder in listdir(csv_location) for file_location in listdir(join(csv_location, folder))]

In [17]:
graph_location = join(data_location, "graphs")
if not exists(graph_location):
    mkdir(graph_location)

distribution_location = join(graph_location, "distribution")
if not exists(distribution_location):
    mkdir(distribution_location)

In [18]:
def get_distributions(folder, location, name):
    frame = pd.read_csv(join(csv_location, folder, name))

    numerical = frame.select_dtypes(include=np.number)
    categorical = frame.select_dtypes(exclude=np.number)

    for column in numerical.columns:
        try:
            fig, (ax1, ax2) = plt.subplots(1, 2)
            ax1.set_title("Skewness:" + str(round(numerical[column].skew(), 2)))
            sns.distplot(numerical[column], ax=ax1)
            ax1.set_ylabel('count')
            sns.boxplot(x=numerical[column], ax=ax2)
            fig.savefig(join(location, "numerical", column) + ".jpg", dpi=400, format="jpeg")
        except:
            continue
        finally: 
            plt.close(fig)
    for column in categorical.columns:
        try:
            fig, ax = plt.subplot(1, 1)
            plt.set_title()
            sns.countplot(categorical, x=column, ax=ax)
            fig.savefig(join(location, "categorical", column) + ".jpg", dpi=400, format="jpeg")
        except:
            continue
        finally: 
            plt.close(fig)

In [19]:
threads = []

for folder, name in files:
    if not exists(join(distribution_location, folder)):
        mkdir(join(distribution_location, folder))
    try:
        number = name.split("_")[1].split(".")[0]
        # number = name.split(".")[0]
        file_location = join(distribution_location, folder, number)
        if not exists(file_location):
            mkdir(file_location)
            mkdir(join(file_location, "numerical"))
            mkdir(join(file_location, "categorical"))

        x = threading.Thread(target=get_distributions, args=(folder, file_location, name))
        threads.append(x)
        x.start()
    except:
        continue

for thread in threads:
    thread.join()